<h1> Recomendador de Filmes </h1>

In [1]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import scipy.sparse as sps
from sklearn.metrics.pairwise import cosine_similarity
import math
#loading dataframes
dfRatings = pd.read_csv('./csv/preProcessedRatings.csv')
dfPretty = pd.read_csv('./csv/contentPrettyDf.csv')
dfContent = pd.read_csv('./csv/contentDf.csv')
dfRatings = dfRatings.drop(columns=['Unnamed: 0'])
dfPretty = dfPretty.drop(columns=['Unnamed: 0'])
dfContent = dfContent.drop(columns=['Unnamed: 0'])
links = pd.read_csv('./csv/links.csv')
links = links.dropna()
links = links.drop(links[links.duplicated(subset=['tmdbId'])].index) #droping duplicated tmdb indices
moviesId = dfContent['id'].values
links = links[links['tmdbId'].isin(moviesId)]

In [23]:
#ratings = [[-1, 862, 4], [-1, 9806, 5], [-1, 109445, 2], [-1, 155, 3]]
ratings = [[5179, 13183, 4], [5179, 9806, 5], [5179, 862, 5], [5179, 10195, 2], [5179, 118340, 3], [5179, 155, 4], [5179, 17473, 1]]
#ratings = [[5179, 13183, 2], [5179, 9806, 0.5], [5179, 862, 0.5], [5179, 10195, 4], [5179, 118340, 3], [5179, 155, 1], [5179, 17473, 5]]

In [2]:
def watchStatus(ratings, dfR):
    #storing movies notWatcheds for collaborative operations
    watched = [i[1] for i in ratings]
    notWatched = list(filter(lambda a: a not in watched, dfR['movieId'].values))
    notWatched = list(dict.fromkeys(notWatched))
    return watched, notWatched

In [3]:
def collaborative(ratings, notWatched, dfR):
    df = dfR.copy()
    actualIndex = df.shape[0] #geting the lenght of the dataframe
    for rating in ratings:
        df.loc[actualIndex] = rating
        actualIndex += 1
    df = df.sort_values(by=['userId'])
    df = df.reset_index().drop(columns=['index'])
    reader = Reader()
    data = Dataset.load_from_df(df, reader)
    trainset = data.build_full_trainset()
    # We'll use the SVD algorithm.
    algo = SVD()
    # Train the algorithm on the trainset, and predict ratings for the testset
    algo.fit(trainset)

    preds = []
    for movieId in notWatched:
        preds.append((ratings[0][0], movieId))
    
    for i in range(len(preds)):
        pred = algo.predict(preds[i][0], preds[i][1])
        preds[i] = (preds[i][1], pred[3]) #geting rating
    
    return preds

<strong> Antes de tudo é necessário reunir as colunas budget, popularity e runtime como atributos numéricos e criar uma matriz esparsa sobre os metadados em string </strong>

In [4]:
#transforming stringMetadata in a matrix with each column a word of the vocabulary
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words='english', min_df=1)
matrix = vect.fit_transform(dfContent['stringMetadata'])
sdf = pd.DataFrame.sparse.from_spmatrix(matrix, columns=vect.get_feature_names(), index=dfContent.index)

sdf

,01distribution,02entertainment,100,1000volt,100bares,100doo,100feetproductions,1019entertainment,101ststreetfilms,106,...,씨너스엔터테인먼트,영화사수박,영화사집,이디오플랜,인벤트디,조병만,진모영,타임스토리그룹,팔레트픽처스,프로덕션m
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#joining the number fields with the string metadata dataframe
numberDf = dfContent[['budget', 'popularity', 'runtime']]
numberValues = numberDf.values

In [ ]:
import numpy as np
ssdf = sdf.to_sparse()
sMatrix = ssdf.to_coo()

def stringSimilarities(index):
    questionMatrix = sps.coo_matrix(sdf.iloc[index].values)
    sparseMatrix = sMatrix.astype(float)
    questionMatrix = questionMatrix.astype(float)
    similarities = cosine_similarity(questionMatrix, sMatrix)
    return similarities

In [7]:
def numberSimilarities(index):
    questionMatrix = [numberDf.iloc[index].values]
    similarities = cosine_similarity(questionMatrix, numberValues)
    return similarities

In [8]:
def contentBased(ratings, notWatched=None):
    preds = []
    if notWatched == None:
        for i in range(len(dfContent)):
            preds.append((dfContent['id'].iloc[i], 0))
    else:
        for ID in notWatched:
            preds.append((ID, 0))
    

    for rating in ratings:
        ratingFactor = float(rating[2]) 
        index = dfContent.index[dfContent['id'] == rating[1]][0]
        ss = stringSimilarities(index)[0]
        ns = numberSimilarities(index)[0]
        if ratingFactor >= 3:
            for i in range(len(preds)):
                index = dfContent.index[dfContent['id'] == preds[i][0]][0]
                preds[i] = (preds[i][0], preds[i][1]+(((6*ss[index] + 3*ns[index])/9)*(ratingFactor)))
        else:
            ratingFactor = 5-ratingFactor
            for i in range(0, len(preds)):
                index = dfContent.index[dfContent['id'] == preds[i][0]][0]
                preds[i] = (preds[i][0], preds[i][1]+((1-((6*ss[index] + 3*ns[index])/9))*(ratingFactor))) #if rating it's bellow 60% get the similarities opposit of the movie 
    for i in range(len(preds)):
        preds[i] = (preds[i][0], (preds[i][1]/len(ratings))) #average in a scale of 0 - 5
    return preds

In [9]:
def filterContentPreds(preds, watched):
    result = [i for i in preds if i[0] not in watched]
    return result

In [10]:
def alterScale(preds, newMin, newMax):
    oldMax = max(preds, key=lambda tup: tup[1])[1]
    oldMin = min(preds, key=lambda tup: tup[1])[1]
    oldRange = oldMax - oldMin
    newRange = newMax - newMin
    newScalePreds = []
    if(oldRange > 0):
        for pred in preds:
            newScaleScore = ((newRange * ((pred[1] - oldMin)) / oldRange)) + newMin
            newScalePreds.append((pred[0], newScaleScore))
    else:
        newScalePreds = preds
    return newScalePreds

In [11]:
def recommender(ratings, dfR, testSetIds=None):
    #collaborative
    watched, notWatched = watchStatus(ratings, dfR)
    notWatched = testSetIds if testSetIds != None else notWatched
    collabPreds = collaborative(ratings, notWatched, dfR)
    
    #content based
    contentPreds = contentBased(ratings, testSetIds)
    
    #manipulating to join content based with collaborative
    contentPreds = filterContentPreds(contentPreds, watched)

    #setting content and collab to same scale
    minCollab = min(collabPreds, key=lambda tup: tup[1])[1]
    maxCollab = max(collabPreds, key=lambda tup: tup[1])[1]
    contentPreds = alterScale(contentPreds, minCollab, maxCollab)
    contentPreds.sort(key=lambda tup: tup[0])
    collabPreds.sort(key=lambda tup: tup[0])
    
    #joining content and collabPreds
    resultPreds = []
    j = 0
    for i in range(len(contentPreds)):
        currentPred = None
        if((j < len(collabPreds)) and (collabPreds[j][0] == contentPreds[i][0])): #if the next pred has in both preds
            currentPred = (contentPreds[i][0], (collabPreds[j][1] + contentPreds[i][1])/2) #mean
            j += 1
        else:
            currentPred = contentPreds[i]
        resultPreds.append(currentPred)
    return resultPreds, contentPreds, collabPreds

In [24]:
resultPreds, contentPreds, collabPreds = recommender(ratings, dfRatings)

In [27]:
getTopNMovies(resultPreds, 30)

828                                          The Godfather
1175                                The Godfather: Part II
2499                                            The Castle
2993                                           Toy Story 2
314                               The Shawshank Redemption
15344                                          Toy Story 3
26774                                        The Happy Elf
1187                                             The Sting
15884                                       Big Buck Bunny
15232                                         Halo Legends
24272                              The Lady and the Reaper
39720                       Starchaser: The Legend of Orin
874                                  It Happened One Night
5480                                         Spirited Away
44792                                             Delivery
6235                                          Finding Nemo
49                                      The Usual Suspec

In [12]:
def getTopNMovies(preds, n):
    preds.sort(key=lambda tup: tup[1], reverse=True)
    scores = [i[0] for i in preds[:n]]
    movieIdx = [dfPretty.loc[dfPretty['id'] == i].index[0] for i in scores]
    return dfPretty['title'].iloc[movieIdx]

#idRatings = [i[1] for i in ratings]
#test = [i for i in contentPreds if i[0] not in idRatings]
#getTopNMovies(test, 30)

In [13]:
def create_testset(user): #creating a trainset of 75% 
    dfR = dfRatings.copy()
    scores = dfR.values
    userScoresLen = sum(1 for score in scores if score[0] == user)
    trainSetLen = userScoresLen*0.75
    trainSet = []
    testSet = []
    found = False
    for i in range(len(scores)):
        while(scores[i][0] == user):
            if(len(trainSet) < trainSetLen):
                trainSet.append(scores[i])
            else:
                testSet.append(scores[i])
            dfR = dfR.drop(i)
            i += 1
            found = True
        if found:
            #stopping loop what we need it's already done
            break
    dfR = dfR.reset_index().drop(columns=['index'])
    return trainSet, testSet, dfR

In [ ]:
import math
import random

def testRMSE(n):
    users = []
    for rating in dfRatings.values:
        if(rating[0] not in users):
            users.append(rating[0])
    numerator = 0 #RMSE numerator
    numeratorCol = 0
    numeratorCont = 0
    denominator = 0 #RMSE denominator
    random.shuffle(users)
    users = users[:n]
    print(users)
    for user in users:
        trainSet, testSet, dfR = create_testset(user)
        userNumerator = 0
        userNumeratorCol = 0
        userNumeratorCont = 0
        userDenominator = 0
        if(len(testSet) > 0):
            testSetIds = [i[1] for i in testSet]
            preds, contentPreds, collabPreds = recommender(trainSet, dfR, testSetIds)
            testSet.sort(key=lambda tup: tup[1])
            for i in range(len(testSet)):
                userNumerator += (preds[i][1] - testSet[i][2])**2
                userNumeratorCol += (collabPreds[i][1] - testSet[i][2])**2
                userNumeratorCont += (contentPreds[i][1] - testSet[i][2])**2
                userDenominator += 1

            numerator += userNumerator
            numeratorCol += userNumeratorCol
            numeratorCont += userNumeratorCont
            denominator += userDenominator
            print("hibrid: ", math.sqrt(userNumerator/userDenominator))
            print("Collaborative", math.sqrt(userNumeratorCol/userDenominator))
            print("Content Based", math.sqrt(userNumeratorCont/userDenominator))
            print(userDenominator)
        print('.....')
    print("hibrid: ", math.sqrt(numerator/denominator))
    print("Collaborative", math.sqrt(numeratorCol/denominator))
    print("Content Based", math.sqrt(numeratorCont/denominator))

testRMSE(10)